In [1]:
# STEP 1 -From Json to YOLO (Roads' Lanes) for training 
import os
import glob
import json
from PIL import Image
import shutil
import random

class_mapping ={
    'L10-1': 0,
    'L18- 1': 1,
    'L2-1': 2,
    'L4-1': 3,
    'L5-1': 4,
    'L6-1b': 5,
    'L7-2': 6,
    'RGA-1': 7,
    'RGA-4': 8,
    'ZigZag Marking': 9
}
class_mapping_1 = {
    "L1-1a": 0,
    "L1-1b": 1,
    "L1-1c": 2,
    "L2-1a": 3,
    "L2-1b": 4,
    "L2-1c": 5,
    "L3-1a": 6,
    "L3-1b": 7,
    "L3-2a": 8,
    "L3-2b": 9,
    "L4-1a": 10,
    "L4-1b": 11,
    "L4-1c": 12,
    "L4-1d": 13,
    "L5-1a": 14,
    "L5-1b": 15,
    "L5-1c": 16,
    "L6-1a": 17,
    "L6-1b": 18,
    "L6-1c": 19,
    "L6-1d": 20,
    "L7-1": 21,
    "L7-2": 22,
    "L8-1": 23,
    "L9-1a": 24,
    "L9-1b": 25,
    "L9-1c": 26,
    "L9-1d": 27,
    "L10-1": 28,
    "L11-1": 29,
    "L18- 1": 30,
    "L21-1": 31,
    "L24-1": 32,
    "L25-1": 33,
    "L26-1": 34,
    "L27-1": 35,
    "L36-1a": 36,
    "L36-1b": 37,
    "L36-1c": 38
}

def convert_to_yolo(json_path, output_dir, image_dir, class_mapping):
    os.makedirs(output_dir, exist_ok=True)
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    # Extract image path
    image_path = os.path.join(image_dir, os.path.basename(data['imagePath']))
    
    # Check if the image exists
    if not os.path.exists(image_path):
        print(f"Warning: Image {image_path} not found. Skipping.")
        return
    
    # Get image dimensions
    with Image.open(image_path) as img:
        image_width, image_height = img.size
    
    # Process annotations in the "shapes" key
    for annotation in data['shapes']:
        label = annotation['label']
        points = annotation['points']
        
        # Skip labels not in the mapping
        if label not in class_mapping:
            continue
        class_id = class_mapping[label]
        
        # Normalize polygon points
        normalized_points = [
            [x / image_width, y / image_height] for x, y in points
        ]
        
        # Flatten points into YOLOv8 format
        flattened_points = " ".join(f"{x} {y}" for x, y in normalized_points)
        yolo_line = f"{class_id} {flattened_points}"
        
        # Write to label file
        shutil.copy(image_path, output_dir)
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        label_file = os.path.join(output_dir, f"{image_name}.txt")
        with open(label_file, 'a') as f:
            f.write(yolo_line + "\n")


allfiles = glob.glob(r'C:\Users\fbpza\Desktop\Road-Lines-Detection\KSA dataset Insp\Annotations' + os.sep + '*.json')
random.shuffle(allfiles)

# Split into 80% train and 20% val
split_index = int(len(allfiles) * 0.8)
train_files = allfiles[:split_index]
val_files = allfiles[split_index:]

# Define output directories
train_output_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\KSA dataset Insp\final\train'
val_output_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\KSA dataset Insp\final\val'
image_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\KSA dataset Insp\Annotations'

# Create directories if they don't exist
os.makedirs(train_output_dir, exist_ok=True)
os.makedirs(val_output_dir, exist_ok=True)

# Process training files
for fileee in train_files:
    convert_to_yolo(
        json_path=fileee,
        output_dir=train_output_dir,
        image_dir=image_dir,
        class_mapping=class_mapping
    )

# Process validation files
for fileee in val_files:
    convert_to_yolo(
        json_path=fileee,
        output_dir=val_output_dir,
        image_dir=image_dir,
        class_mapping=class_mapping
    )



In [ ]:
# STEP:2 For Annotations(with drawings and yolo_V8 annotations) for both images and videos
import cv2
import numpy as np
import glob, os
from shutil import copyfile
from ultralytics import YOLO

def calculate_iou(box1, box2):
    x1, y1, x2, y2 = box1
    x1_, y1_, x2_, y2_ = box2

    xi1 = max(x1, x1_)
    yi1 = max(y1, y1_)
    xi2 = min(x2, x2_)
    yi2 = min(y2, y2_)

    inter_area = max(0, xi2 - xi1) * max(0, yi1 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2_ - x1_) * (y2_ - y1_)

    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area
    return iou

lines_model = 'Models/Lanes_seg_v2.pt'
model = YOLO(lines_model)

counter = 0
images_src_dir = 'Data/Raw_images'
images_list = [i for i in glob.glob(images_src_dir + os.sep + "*.jpg")]
videos_list = [i for i in glob.glob(images_src_dir + os.sep + "*.mp4")]
annotated_output_dir = 'Data/Detected/annotated_images'
original_output_dir = 'Data/Detected/original_images_with_annotations'

os.makedirs(annotated_output_dir, exist_ok=True)
os.makedirs(original_output_dir, exist_ok=True)

colors = [
    (255, 0, 0),  # Red
    (255, 0, 0),  # Green
    (0, 0, 255),  # Blue
    (255, 255, 0),  # Cyan
    (255, 0, 255),  # Magenta
    (0, 255, 255),  # Yellow
]

overlap_threshold = 0.5  # Define the overlap threshold

# Process images
for imagefile in images_list:
    image_name = os.path.basename(imagefile)
    image_path = imagefile

    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]  # Get image dimensions

    results = model(img, conf=0.3)

    blended_image = img.copy()  # Create a copy of the original image for blending
    masks_detected = False  # Flag to check if any masks were detected

    filtered_results = []
    yolo_format_lines = []  # List to store YOLOv8 format lines

    for idx, result in enumerate(results):
        if result.masks is not None:
            boxes = result.boxes.xyxy.cpu().numpy().astype(int)
            confidences = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy().astype(int)
            class_names = result.names  # Get class names
            # Filter out overlapping boxes
            keep = np.ones(len(boxes), dtype=bool)
            for i in range(len(boxes)):
                if not keep[i]:
                    continue
                for j in range(i + 1, len(boxes)):
                    if calculate_iou(boxes[i], boxes[j]) > overlap_threshold:
                        if confidences[i] > confidences[j]:
                            keep[j] = False
                        else:
                            keep[i] = False

            filtered_boxes = boxes[keep]
            filtered_confidences = confidences[keep]
            filtered_classes = classes[keep]
            filtered_masks = result.masks.data[keep]

            filtered_results.append((filtered_boxes, filtered_confidences, filtered_classes, filtered_masks))

    for boxes, confidences, classes, masks in filtered_results:
        for i, mask in enumerate(masks):
            masks_detected = True  # Set flag to True if any mask is detected
            mask = (mask.cpu().numpy() * 255).astype("uint8")  # Move tensor to CPU and convert to binary
            mask_resized = cv2.resize(mask, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
            
            # Find contours of the mask
            contours, _ = cv2.findContours(mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            if contours:
                # Approximate contours to reduce the number of points
                approx_contours = [cv2.approxPolyDP(cnt, epsilon=1.0, closed=True) for cnt in contours]
                
                for cnt in approx_contours:
                    for j in range(len(cnt)):
                        point = cnt[j][0]
                        distance_from_center = abs(point[0] - img_width / 2)
                        max_distance = img_width / 2
                        shift_factor = (distance_from_center / max_distance) * 38  # Gradient shift

                        if point[0] < img_width / 2:
                            # Point is on the left side of the image
                            shift_x = shift_factor
                        else:
                            # Point is on the right side of the image
                            shift_x = -shift_factor

                        shift_y = 0  # No vertical shift needed, adjust if necessary
                        M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
                        shifted_point = cv2.transform(np.array([[point]], dtype=np.float32), M)[0][0]
                        cnt[j][0] = shifted_point

                    # Connect endpoints of the contours to handle discontinuities
                    for j in range(len(cnt) - 1):
                        cv2.line(blended_image, tuple(cnt[j][0]), tuple(cnt[j + 1][0]), (0, 255, 0), 1)
                    cv2.line(blended_image, tuple(cnt[-1][0]), tuple(cnt[0][0]), (0, 255, 0), 1)
                
                class_name = class_names[classes[i]]
                color = colors[classes[i] % len(colors)]  # Get color for the class

                cv2.putText(
                    blended_image,
                    class_name,
                    (point[0], point[1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    color,
                    2,
                    cv2.LINE_AA,
                )

                # Convert to YOLOv8 format
                x1, y1, x2, y2 = boxes[i]
                bbox_width = x2 - x1
                bbox_height = y2 - y1
                yolo_center_x = (x1 + x2) / 2 / img_width
                yolo_center_y = (y1 + y2) / 2 / img_height
                yolo_width = bbox_width / img_width
                yolo_height = bbox_height / img_height

                # Get the segmentation points
                for contour in contours:
                    points = contour.reshape(-1, 2)
                    normalized_points = [(x / img_width, y / img_height) for x, y in points]
                    formatted_points = " ".join([f"{x} {y}" for x, y in normalized_points])
                    yolo_format_lines.append(f"{classes[i]} {formatted_points}")

    if masks_detected:
        # Save the annotated image
        annotated_output_path = os.path.join(annotated_output_dir, f"{os.path.splitext(image_name)[0]}.jpg")
        cv2.imwrite(annotated_output_path, blended_image)

        # Save YOLOv8 format annotations
        txt_output_path = os.path.join(original_output_dir, f"{os.path.splitext(image_name)[0]}.txt")
        with open(txt_output_path, 'w') as f:
            f.write("\n".join(yolo_format_lines))
        
        # Copy the original image to the original output directory
        copyfile(image_path, os.path.join(original_output_dir, image_name))
        
        counter += 1
    else:
        print(f"No masks detected for image: {image_name}")

# Process videos
for videofile in videos_list:
    video_name = os.path.basename(videofile)
    input_video_path = videofile
    output_video_path = os.path.join(annotated_output_dir, f"{os.path.splitext(video_name)[0]}_resulted.mp4")

    cap = cv2.VideoCapture(input_video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    frame_counter = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        img_height, img_width = frame.shape[:2]  # Get frame dimensions

        results = model(frame, conf=0.3)

        blended_image = frame.copy()  # Create a copy of the original frame for blending
        masks_detected = False  # Flag to check if any masks were detected

        filtered_results = []
        yolo_format_lines = []  # List to store YOLOv8 format lines

        for idx, result in enumerate(results):
            if result.masks is not None:
                boxes = result.boxes.xyxy.cpu().numpy().astype(int)
                confidences = result.boxes.conf.cpu().numpy()
                classes = result.boxes.cls.cpu().numpy().astype(int)
                class_names = result.names  # Get class names

                # Filter out overlapping boxes
                keep = np.ones(len(boxes), dtype=bool)
                for i in range(len(boxes)):
                    if not keep[i]:
                        continue
                    for j in range(i + 1, len(boxes)):
                        if calculate_iou(boxes[i], boxes[j]) > overlap_threshold:
                            if confidences[i] > confidences[j]:
                                keep[j] = False
                            else:
                                keep[i] = False

                filtered_boxes = boxes[keep]
                filtered_confidences = confidences[keep]
                filtered_classes = classes[keep]
                filtered_masks = result.masks.data[keep]

                filtered_results.append((filtered_boxes, filtered_confidences, filtered_classes, filtered_masks))

        for boxes, confidences, classes, masks in filtered_results:
            for i, mask in enumerate(masks):
                masks_detected = True  # Set flag to True if any mask is detected
                mask = (mask.cpu().numpy() * 255).astype("uint8")  # Move tensor to CPU and convert to binary
                mask_resized = cv2.resize(mask, (img_width, img_height), interpolation=cv2.INTER_NEAREST)
                
                # Find contours of the mask
                contours, _ = cv2.findContours(mask_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                
                if contours:
                    # Approximate contours to reduce the number of points
                    approx_contours = [cv2.approxPolyDP(cnt, epsilon=1.0, closed=True) for cnt in contours]
                    
                    for cnt in approx_contours:
                        for j in range(len(cnt)):
                            point = cnt[j][0]
                            distance_from_center = abs(point[0] - img_width / 2)
                            max_distance = img_width / 2
                            shift_factor = (distance_from_center / max_distance) * 38  # Gradient shift

                            if point[0] < img_width / 2:
                                # Point is on the left side of the image
                                shift_x = shift_factor
                            else:
                                # Point is on the right side of the image
                                shift_x = -shift_factor

                            shift_y = 0  # No vertical shift needed, adjust if necessary
                            M = np.float32([[1, 0, shift_x], [0, 1, shift_y]])
                            shifted_point = cv2.transform(np.array([[point]], dtype=np.float32), M)[0][0]
                            cnt[j][0] = shifted_point

                        # Connect endpoints of the contours to handle discontinuities
                        for j in range(len(cnt) - 1):
                            cv2.line(blended_image, tuple(cnt[j][0]), tuple(cnt[j + 1][0]), (0, 255, 0), 1)
                        cv2.line(blended_image, tuple(cnt[-1][0]), tuple(cnt[0][0]), (0, 255, 0), 1)
                
                class_name = class_names[classes[i]]
                color = colors[classes[i] % len(colors)]  # Get color for the class

                cv2.putText(
                    blended_image,
                    class_name,
                    (point[0], point[1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    color,
                    2,
                    cv2.LINE_AA,
                )

                # Convert to YOLOv8 format
                x1, y1, x2, y2 = boxes[i]
                bbox_width = x2 - x1
                bbox_height = y2 - y1
                yolo_center_x = (x1 + x2) / 2 / img_width
                yolo_center_y = (y1 + y2) / 2 / img_height
                yolo_width = bbox_width / img_width
                yolo_height = bbox_height / img_height

                # Get the segmentation points
                for contour in contours:
                    points = contour.reshape(-1, 2)
                    normalized_points = [(x / img_width, y / img_height) for x, y in points]
                    formatted_points = " ".join([f"{x} {y}" for x, y in normalized_points])
                    yolo_format_lines.append(f"{classes[i]} {formatted_points}")

        if masks_detected:
            out.write(blended_image)
        else:
            out.write(frame)

        # Save the original frame and YOLOv8 format annotations
        if masks_detected:
            frame_output_path = os.path.join(original_output_dir, f"{os.path.splitext(video_name)[0]}_frame_{frame_counter}.jpg")
            cv2.imwrite(frame_output_path, frame)

            txt_output_path = os.path.join(original_output_dir, f"{os.path.splitext(video_name)[0]}_frame_{frame_counter}.txt")
            with open(txt_output_path, 'w') as f:
                f.write("\n".join(yolo_format_lines))

        frame_counter += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [30]:
# STEP 3 -Encoding images to base64 in JSON files + TXT to json and removing txt files
import os
import glob
import json
from PIL import Image
import base64

class_mapping = {
    'L10-1': 0,
    'L18- 1': 1,
    'L2-1': 2,
    'L4-1': 3,
    'L5-1': 4,
    'L6-1b': 5,
    'L7-2': 6,
    'RGA-1': 7,
    'RGA-4': 8,
    'ZigZag Marking': 9
}

# Define the additional keys and their values
version = "5.5.0"
flags = {}
group_id = 346
description = ""
shape_type = "polygon"
mask = None

def yolo_to_json(yolo_data):
    shapes = []
    for data in yolo_data:
        shape = {
            "label": data["label"],
            "points": data["points"],
            "group_id": group_id,
            "description": description,
            "shape_type": shape_type,
            "flags": flags,
            "mask": mask
        }
        shapes.append(shape)
    
    json_data = {
        "version": version,
        "flags": flags,
        "shapes": shapes
    }
    
    return json_data

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')
    return encoded_string

def convert_to_json(txt_path, output_dir, image_dir, class_mapping):
    os.makedirs(output_dir, exist_ok=True)
    
    with open(txt_path, 'r') as f:
        lines = f.readlines()

    image_name = os.path.splitext(os.path.basename(txt_path))[0]
    image_path = os.path.join(image_dir, f"{image_name}.jpg")
    
    if not os.path.exists(image_path):
        print(f"Warning: Image {image_path} not found. Skipping.")
        return
    with Image.open(image_path) as img:
        image_width, image_height = img.size
    
    yolo_data = []
    
    # Process each line in the YOLOv8 format file
    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        points = list(map(float, parts[1:]))
        
        # Denormalize points
        denormalized_points = [
            [points[i] * image_width, points[i + 1] * image_height] for i in range(0, len(points), 2)
        ]
        
        # Get label from class mapping
        label = [k for k, v in class_mapping.items() if v == class_id][0]
        
        # Add annotation to YOLO data structure
        yolo_data.append({
            "label": label,
            "points": denormalized_points
        })
    
    # Convert YOLO data to JSON format
    json_data = yolo_to_json(yolo_data)
    json_data["imagePath"] = image_name + ".jpg"
    
    # Encode image to base64 and add to JSON data
    encoded_image = encode_image_to_base64(image_path)
    json_data["imageData"] = encoded_image
    
    # Save JSON file
    json_output_path = os.path.join(output_dir, f"{image_name}.json")
    with open(json_output_path, 'w') as f:
        json.dump(json_data, f, indent=4)
    
    # Remove the original txt file
    os.remove(txt_path)

files_dir = r'C:\Users\fbpza\Desktop\Road-Lines-Detection\KSA dataset Insp\final\train'
allfiles = glob.glob(files_dir + os.sep + '*.txt')

# Process training files
for fileee in allfiles:
    convert_to_json(fileee, files_dir, files_dir, class_mapping)